In [1]:
import pandas as pd
import numpy as np
import requests
from time import sleep
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

In [2]:
bing_maps_api_key = "bing_api_hey"

In [3]:
def get_clinic_data(page_number):
    url = f"https://www.local.ch/en/q/Switzerland/clinique?page={page_number}&rid=j6VE&slot=tel"
    global response
    response = requests.get(url, timeout = 20)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        listings_ = soup.find_all("div", class_ = "entry-card lui-padding-horizontal-m lui-padding-vertical-m entry-card-highlight entry-card-tel")
        listings.extend(listings_)
        
listings = []
page_number = 1 
    
while True:
    get_clinic_data(page_number)
    print(f'Page {page_number} completed')
    page_number += 1
    if response.status_code != 200:
        break

Page 1 completed
Page 2 completed
Page 3 completed
Page 4 completed
Page 5 completed
Page 6 completed
Page 7 completed
Page 8 completed
Page 9 completed
Page 10 completed
Page 11 completed
Page 12 completed
Page 13 completed
Page 14 completed
Page 15 completed
Page 16 completed
Page 17 completed
Page 18 completed
Page 19 completed
Page 20 completed
Page 21 completed
Page 22 completed
Page 23 completed
Page 24 completed
Page 25 completed
Page 26 completed
Page 27 completed
Page 28 completed
Page 29 completed
Page 30 completed
Page 31 completed
Page 32 completed
Page 33 completed
Page 34 completed
Page 35 completed
Page 36 completed
Page 37 completed
Page 38 completed
Page 39 completed
Page 40 completed
Page 41 completed
Page 42 completed
Page 43 completed
Page 44 completed
Page 45 completed
Page 46 completed
Page 47 completed
Page 48 completed
Page 49 completed
Page 50 completed
Page 51 completed
Page 52 completed
Page 53 completed
Page 54 completed
Page 55 completed
Page 56 completed
P

In [4]:
clinic_names = []
for names in listings:
    try:
        name_element = names.find("h2", class_="lui-margin-vertical-zero card-info-title")
        name = name_element.text.strip()
        clinic_names.append(name)
    except AttributeError:
        continue
len(clinic_names)

249

In [5]:
clinic_address = []
for address in listings:
    ad = address.find('div', class_="card-info-address")
    clinic_address.append(ad.text.strip())
len(clinic_address)

249

In [6]:
type_of_clinic = []
for type_ in listings:
    ty = type_.find('div',  class_ = "card-info-category")
    type_of_clinic.append(ty.text.split()[0])
len(type_of_clinic) 

249

In [7]:
review = []
for r in listings:
    ar = r.find('span', class_ = "summary-ratings-average lui-font-bold")
    if ar:
        review.append(float(ar.text))
    else:
        review.append(ar)
len(review)

249

In [8]:
rating = []
for l in listings:
    aa = l.find('div', class_ = "summary-ratings-count" )
    try:
        if aa:
            rating.append(int(aa.text.split('\n(')[1].split(')\n')[0]))
        else:
            rating.append(aa)
    except IndexError:
        rating.append(aa)
len(rating)

249

In [9]:
df = pd.DataFrame({'Names':clinic_names,
                    'Type':type_of_clinic,
                    'Address':clinic_address,
                    'Reviews':review,
                    'Rating':rating})

df = df.reset_index(drop=True)
df.index += 1


In [10]:
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')


In [11]:
df['Reviews'] = pd.to_numeric(df['Reviews'], errors = 'coerce')

In [12]:
addresses = df.Address.to_list()

In [13]:
# Replace YOUR_BING_MAPS_API_KEY with your actual Bing Maps API key
lat = []
long = []

# Address to be geocoded
for address in addresses:

    # Construct the URL for the API request
    url = f"http://dev.virtualearth.net/REST/v1/Locations?q={address}&key={bing_maps_api_key}"

    # Make the API request
    response = requests.get(url)

    # Check the status code of the response
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Check if there are any results in the response
        if data["resourceSets"][0]["estimatedTotal"] > 0:
            # Get the first result
            result = data["resourceSets"][0]["resources"][0]

            # Extract the latitude and longitude
            latitude = result["point"]["coordinates"][0]
            longitude = result["point"]["coordinates"][1]
            lat.append(latitude)
            long.append(longitude)
        else:
            print("No results found")
    else:
        print("Request failed with status code:", response.status_code)


In [14]:
df['Longitude'], df['Latitude'] = long, lat

In [15]:
columns = ['Names', 'Type', 'Address', 'Latitude', 'Longitude', 'Reviews', 'Rating' ]

In [16]:
df = df.reindex(columns = columns)

In [17]:
df.to_csv('Swiz_clinc.csv', index = False)